In [13]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand((2, 4))
net(X)

tensor([[-0.4246],
        [-0.4319]], grad_fn=<AddmmBackward>)

## 参数访问

In [14]:
# 通过索引访问任意层的参数
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.0340,  0.2614, -0.0575, -0.3394, -0.2215, -0.2271,  0.3302,  0.2944]])),
             ('bias', tensor([-0.1902]))])

In [26]:
# 访问所有的参数
print(*[(name, param.shape) for name, param in net[2].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([1, 8])) ('bias', torch.Size([1]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [29]:
# 获取参数底层数值
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net.state_dict()['2.bias'].data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1902], requires_grad=True)
tensor([-0.1902])
tensor([-0.1902])


In [30]:
# 在没有调用反向传播前，参数的梯度处于初始状态
net[2].weight.grad == None

True

In [33]:
# 从嵌套块中获取参数

def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2650],
        [-0.2651]], grad_fn=<AddmmBackward>)

In [36]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [40]:
rgnet[0][1][0].bias.data

tensor([-0.4172, -0.0313, -0.2488,  0.0774,  0.2440, -0.0368,  0.2785,  0.3188])

## 参数初始化

In [41]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [44]:
# 使用内置的初始化器，将所有权重参数初始化为标准差为0.01的高斯分布，偏置设置为0
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0] 

(tensor([0.0028, 0.0044, 0.0169, 0.0082]), tensor(0.))

In [46]:
# 将参数初始化为常数
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [47]:
# 应用不同的初始化方法
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.2414,  0.6705,  0.0257,  0.5948])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U(-10, -5) & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$

In [49]:
def my_init(m):
    if type(m) == nn.Linear:
        print('Init',
             *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() > 5
        
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-8.5459, -0.0000, -0.0000,  6.6380],
        [-0.0000,  7.4844, -5.6614,  0.0000]], grad_fn=<SliceBackward>)

## 参数绑定

In [50]:
# 共享参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), 
                    shared, nn.ReLU(), nn.Linear(8,1))
net(X)

tensor([[0.0607],
        [0.0569]], grad_fn=<AddmmBackward>)

In [54]:
# 第二层和第三层的参数时绑定的，改变其中一个值，另一个也会改变
id(net[2]) == id(net[4])

True